In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random
import requests
from collections import deque
from tqdm import tqdm
import time

# HERE API Key
HERE_API_KEY = "0FKiA4FiUL0QqYRWD_xhlg_JVxKSzUv7hHDOnUtdWmo"  # Replace with your actual HERE API key

def get_lat_long(address):
    """Fetch latitude and longitude using HERE API."""
    url = f"https://geocode.search.hereapi.com/v1/geocode?q={address}&apiKey={HERE_API_KEY}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        if "items" in data and len(data["items"]) > 0:
            location = data["items"][0]["position"]
            return location["lat"], location["lng"]
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {address}: {e}")
    return None, None

# Fetch latitude and longitude for Chennai locations
chennai_areas = [ "T Nagar, Chennai, India", "Nungambakkam, Chennai, India", "Egmore, Chennai, India", "Purasaiwakkam, Chennai, India", 
    "Kilpauk, Chennai, India", "Adyar, Chennai, India", "Velachery, Chennai, India", "OMR, Chennai, India", "Tambaram, Chennai, India",
    "Thiruvanmiyur, Chennai, India", "Perambur, Chennai, India", "Tondiarpet, Chennai, India", "Royapuram, Chennai, India",
    "Washermanpet, Chennai, India", "Korukkupet, Chennai, India", "Anna Nagar, Chennai, India", "Mogappair, Chennai, India",
    "Maduravoyal, Chennai, India", "Porur, Chennai, India", "Ambattur, Chennai, India",
    "Mylapore, Chennai, India", "Kodambakkam, Chennai, India", "Ashok Nagar, Chennai, India",
    "Vadapalani, Chennai, India", "Aminjikarai, Chennai, India", "Perungudi, Chennai, India", "Sholinganallur, Chennai, India",
    "Pallavaram, Chennai, India", "Chromepet, Chennai, India", "Guindy, Chennai, India", "Vepery, Chennai, India",
    "Saidapet, Chennai, India", "Triplicane, Chennai, India", "Teynampet, Chennai, India", "Kotturpuram, Chennai, India",
    "Thoraipakkam, Chennai, India", "Poonamallee, Chennai, India", "Manapakkam, Chennai, India", "Ramapuram, Chennai, India",
    "Meenambakkam, Chennai, India", "Tiruvottiyur, Chennai, India", "Alandur, Chennai, India", "Ennore, Chennai, India",
    "Minjur, Chennai, India", "Redhills, Chennai, India", "Avadi, Chennai, India", "Pattabiram, Chennai, India",
    "Thirumullaivoyal, Chennai, India", "Villivakkam, Chennai, India", "Thiruverkadu, Chennai, India", "Kundrathur, Chennai, India",
    "AGS Colony, Chennai, India", "Mannivakkam, Chennai, India", "Padappai, Chennai, India", "Madipakkam, Chennai, India",
    "Medavakkam, Chennai, India", "Selaiyur, Chennai, India", "Guduvanchery, Chennai, India", "Kelambakkam, Chennai, India",
    "Navalur, Chennai, India", "Siruseri, Chennai, India", "Urapakkam, Chennai, India", "Vandalur, Chennai, India","Mugalivakkam,Chennai,India",
    "Gerugambakkam,Chennai,India","Kattupakkam,Chennai,India","Iyyapanthangal,Chennai,India","Vanagaram,Chennai,India","ITC Grand Chola,Chennai,India",
    "West Saidapet,Chennai,India","Nandambakkam,Chennai,India","Mannady,Chennai,India","Madhavaram,Chennai,India",
    "Thirumazhisai,Chennai,India","Maduravoyal,Chennai,India","Ashok Nagar,Chennai,India","Ashok Pillar,Chennai,India",
    "Chennai Rail Museum,Chennai,India","Rippon Building,Chennai,India","Stanley Hospital,Chennai,India","Aynavaram,Chennai,India",
    "Purasaivakkam,Chennai,India","Vyasarpadi,Chennai,India","Padi,Chennai,India","Pattalam,Chennai,India","Rohini Silver Screens,Chennai,India","Mugalivakkam,Chennai,India",
    "MMDA Colony,Chennai,India","Government Museum Chennai,Chennai,India", "Arumbakkam,Chennai,India","Thiru Vi Ka Park,Chennai,India"
                ]

tqdm.pandas()
df_chennai = pd.DataFrame(chennai_areas, columns=["Address"])
df_chennai[['Latitude', 'Longitude']] = df_chennai["Address"].progress_apply(lambda x: pd.Series(get_lat_long(x)))


df_chennai.to_csv("chennai_custdata.csv", index=False)
print("\nUpdated dataset saved.\n")

# Load and preprocess data
df = pd.read_csv("chennai_custdata.csv")
df = df.dropna(subset=["Latitude", "Longitude"])

# Normalize coordinates
min_lat, max_lat = df["Latitude"].min(), df["Latitude"].max()
min_lon, max_lon = df["Longitude"].min(), df["Longitude"].max()
df["Latitude"] = (df["Latitude"] - min_lat) / (max_lat - min_lat)
df["Longitude"] = (df["Longitude"] - min_lon) / (max_lon - min_lon)

# Define Deep Q-Network (DQN)
class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, output_dim)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# Define Agent
class Agent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=50000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.999
        self.learning_rate = 0.0005
        self.model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.criterion = nn.MSELoss()
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.randrange(self.action_size)
        state = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            action_values = self.model(state)
        return torch.argmax(action_values).item()
    
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward if done else reward + self.gamma * torch.max(self.model(torch.FloatTensor(next_state).unsqueeze(0))).item()
            target_f = self.model(torch.FloatTensor(state).unsqueeze(0)).clone().detach()
            target_f[0][action] = target
            self.optimizer.zero_grad()
            loss = self.criterion(self.model(torch.FloatTensor(state).unsqueeze(0)), target_f)
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Train the RL Model
def train_rl_model(df, episodes=2000):
    state_size = 2  # Latitude and Longitude
    action_size = len(df)  # Each location as an action
    agent = Agent(state_size, action_size)
    for e in range(episodes):
        state = df.sample(1)[['Latitude', 'Longitude']].values[0]
        done = False
        while not done:
            action = agent.act(state)
            next_state = df.iloc[action][['Latitude', 'Longitude']].values
            reward = -np.linalg.norm(state - next_state)
            done = True
            agent.remember(state, action, reward, next_state, done)
        agent.replay(32)
    return agent

agent = train_rl_model(df, episodes=2000)


100%|██████████████████████████████████████████████████████████████████████████████████| 91/91 [00:24<00:00,  3.79it/s]



Updated dataset saved.



In [2]:
import folium
import pandas as pd

# Load the dataset
df = pd.read_csv("chennai_custdata.csv")

# Initialize a map centered around Chennai
chennai_map = folium.Map(location=[df["Latitude"].mean(), df["Longitude"].mean()], zoom_start=11)

# Add markers for each location
for _, row in df.iterrows():
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=row["Address"],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(chennai_map)

# Save the map
chennai_map.save("chennai_map.html")
print("Map saved as chennai_map.html. Open it in a browser to view.")

Map saved as chennai_map.html. Open it in a browser to view.
